# Synthetic csv generation

In [2]:
import pandas as pd
from datetime import datetime

# Create synthetic data for MainMessages
main_messages_data = {
    'message_id': [1, 2, 3],
    'user_id': ['user1', 'user2', 'user1'],
    'message_content': ['Hello, how can I help you?', 'What are your service hours?', 'Thank you!'],
    'timestamp': [datetime.now(), datetime.now(), datetime.now()],
    'is_user_message': [True, True, True]
}

# Create a DataFrame and save to CSV
main_messages_df = pd.DataFrame(main_messages_data)
main_messages_df.to_csv('MainMessages.csv', index=False)

# Create synthetic data for AgentConversations
agent_conversations_data = {
    'thread_message_id': [1, 2, 3, 4, 5],
    'message_id': [1, 1, 2, 2, 3],
    'agent_name': ['AI-coach', 'AI-psychologist', 'AI-coach', 'AI-psychologist', 'AI-career-advisor'],
    'message_content': [
        'Let’s discuss your goals.', 
        'How do you feel about your current progress?',
        'Our service hours are 9-5, Monday-Friday.', 
        'Is there anything specific you need help with during those hours?',
        'You’re welcome! Good luck!'
    ],
    'timestamp': [datetime.now(), datetime.now(), datetime.now(), datetime.now(), datetime.now()]
}

# Create a DataFrame and save to CSV
agent_conversations_df = pd.DataFrame(agent_conversations_data)
agent_conversations_df.to_csv('AgentConversations.csv', index=False)

# Provide links to download the CSV files
# main_messages_df_path = '/mnt/data/MainMessages.csv'
# agent_conversations_df_path = '/mnt/data/AgentConversations.csv'

# main_messages_df_path, agent_conversations_df_path


In [5]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import pandas as pd
from datetime import datetime

# Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Creates local webserver and auto handles authentication.
drive = GoogleDrive(gauth)

def download_update_upload(file_id, new_data):
    # Download file
    file = drive.CreateFile({'id': file_id})
    file.GetContentFile('file.csv')
    
    # Load to DataFrame, append data, and save
    df = pd.read_csv('file.csv')
    df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)
    df.to_csv('file.csv', index=False)
    
    # Upload the updated file
    file.SetContentFile('file.csv')
    file.Upload()  # Upload the file with the updated content.
# https://drive.google.com/file/d/16Jdb6qXJG7rRO0LKrwTN6t2XgwVIXv-C/view?usp=sharing
# Example usage
file_id = '16Jdb6qXJG7rRO0LKrwTN6t2XgwVIXv-C'
new_data = {
    'message_id': [4],  # Assuming continuation of IDs
    'user_id': ['new_user'],
    'message_content': ['Another test message'],
    'timestamp': [datetime.now()],
    'is_user_message': [True]
}
download_update_upload(file_id, new_data)


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [6]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'orion-431410-ee3221a6bafe.json'

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and construct service
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))


Files:
Orion (1zJve83hWn-8crucPPCZ-DVlziySHejNT)
AgentConversations.csv (16Jdb6qXJG7rRO0LKrwTN6t2XgwVIXv-C)
MainMessages.csv (1ymyv3IOeBX5rCnjSIRGGeEfAmBzmcZG0)


In [7]:
import io
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd

# Service account and Drive setup
SERVICE_ACCOUNT_FILE = 'orion-431410-ee3221a6bafe.json'
SCOPES = ['https://www.googleapis.com/auth/drive']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

# Google Drive file ID for MainMessages.csv
FILE_ID = '1ymyv3IOeBX5rCnjSIRGGeEfAmBzmcZG0'

# Function to download file from Google Drive
def download_file(service, file_id, local_filename):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    with io.open(local_filename, 'wb') as f:
        fh.seek(0)
        f.write(fh.read())

# Function to upload file to Google Drive
def upload_file(service, file_id, local_filename):
    file_metadata = {'name': local_filename}
    media = MediaFileUpload(local_filename, mimetype='text/csv')
    service.files().update(fileId=file_id, body=file_metadata,
                           media_body=media, fields='id').execute()

# Downloading the file from Google Drive
local_filename = "MainMessages.csv"
download_file(service, FILE_ID, local_filename)

# Read, update, and save the CSV file
df = pd.read_csv(local_filename)
new_data = {'message_id': [4], 'user_id': ['user3'], 'message_content': ["Let's update this file!"], 'timestamp': ['2024-08-03 11:20:00.123456'], 'is_user_message': [True]}
new_df = pd.DataFrame(new_data)
df = pd.concat([df, new_df], ignore_index=True)
df.to_csv(local_filename, index=False)

# Upload the updated file back to Google Drive
upload_file(service, FILE_ID, local_filename)
